はじめに
-------

Hyにより Chainerのサンプルを記述します。

Chainerのインストール
-----------------

In [2]:
!pip install chainer

    100% |████████████████████████████████| 348kB 1.2MB/s ta 0:00:011
    100% |████████████████████████████████| 61kB 5.3MB/s eta 0:00:01
    100% |████████████████████████████████| 163kB 3.9MB/s eta 0:00:01
    100% |████████████████████████████████| 102kB 2.2MB/s a 0:00:01
  Running setup.py bdist_wheel for chainer ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/50/a6/5f/1b4d884fcc4e079cfc6f60f1ba0f5d1cffff63be09c111e273
  Running setup.py bdist_wheel for filelock ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/68/e0/0c/ca5c34b3b61ef8d3caf4b7fbbfb2f9f840128ce4015ad52566
Successfully built chainer filelock


Hyで記述した [Chainerのチュートリアル](https://docs.chainer.org/en/stable/tutorial/basic.html#core-concept)を以下に示します。

In [1]:
(import [numpy :as np])
(import chainer)
(import [chainer [cuda Function gradient_check report training utils Variable
                  datasets iterators optimizers serializers Link Chain ChainList]])
(import [chainer.functions :as F]
        [chainer.links :as L])
(import [chainer.training [extensions]])

In [2]:
(def x_data (np.array 5 :dtype np.float32))
(def x (Variable x_data))
(def y (+ (** x 2) (* -2 x) 1))  ;;y = x**2 - 2 * x + 1
(print y.data)

16.0


In [3]:
(y.backward)
(print x.grad)

8.0


In [4]:
(def z (* 2 x))
(def y (+ (** 2 x) (* -1 z) 1))
(.backward y :retain_grad True)
(print z.grad)

-1.0


In [5]:
(def x (Variable (np.array [[1 2 3] [4 5 6]] :dtype np.float32)))
(def y (+ (** 2 x) (* -2 x) 1))
(def y.grad (np.ones [2 3] :dtype np.float32))
(y.backward)
(print x.grad)

[[ -0.61370564   0.77258873   3.54517746]
 [  9.09035492  20.18070984  42.36141968]]


In [6]:
(def f (L.Linear 3 2))
(print f.W.data)
(print f.b.data)

[[-0.1960112   0.00162482 -0.16950487]
 [-0.22733469 -0.23141789 -0.98010153]]
[ 0.  0.]


In [7]:
(def x (Variable (np.array [[1 2 3] [4 5 6]] :dtype np.float32)))
(def y (f x))
(print y.data)

[[-0.70127612 -3.63047504]
 [-1.79294991 -7.94703722]]


In [8]:
(def l1 (L.Linear 4 3))
(def l2 (L.Linear 3 2))
(defn my_forward [x]
  (def h (l1 x))
  (l2 h))

In [9]:
(defclass MyProc [object]
  (defn --init-- [self]
    (def self.l1 (L.Linear 4 3))
    (def self.l2 (L.Linear 3 2)))
  (defn forward [self x]
    (def h (self.l1 x))
    (self.l2 h)))

In [10]:
(defclass MyChain [Chain]
  (defn --init-- [self]
    (.__init__ (super MyChain self))
    (with [(self.init_scope)]
      (def self.l1 (L.Linear 4 3))
      (def self.l2 (L.Linear 3 2))))

  (defn --call-- [self x]
    (def h (self.l1 x))
    (self.l2 h)))

In [11]:
(defclass MyChain2 [ChainList]
  (defn --init-- [self]
    (.__init__ (super MyChain2 self)
                 (L.Linear 4 3)
                 (L.Linear 3 2)))

  (defn --call-- [self x]
    (def h (self[0] x))
    (self[1] h)))

In [12]:
(def model (MyChain))
(def optimizer (optimizers.SGD))
(optimizer.setup model)

In [13]:
(optimizer.add_hook (chainer.optimizer.WeightDecay 0.0005))

In [14]:
(def x (.astype (np.random.uniform -1 1 [2 4]) np.float32))
(model.cleargrads)
;; compute gradient here...
(def loss (F.sum (model (chainer.Variable x))))
(loss.backward)
(optimizer.update)

In [15]:
(defn lossfun [arg1 arg2]
  (def loss (F.sum (model (- arg1 arg2))))
  loss)

(def arg1 (.astype (np.random.uniform -1 1 [2 4]) np.float32))
(def arg2 (.astype (np.random.uniform -1 1 [2 4]) np.float32))
(.update optimizer lossfun (chainer.Variable arg1) (chainer.Variable arg2))

In [16]:
(serializers.save_npz "my.model" model)

In [17]:
(def [train test] (datasets.get_mnist))

In [18]:
(def train_iter (iterators.SerialIterator train :batch_size 100 :shuffle True))

In [24]:
(defclass MLP [Chain]
  (defn --init-- [self n_units n_out]
    (.__init__ (super MLP self))
    (with [(self.init_scope)]
      (def self.l1 (L.Linear None n_units))  ;; n_in -> n_units
      (def self.l2 (L.Linear None n_units))  ;; n_units -> n_units
      (def self.l3 (L.Linear None n_out))))  ;; n_units -> n_out

  (defn --call-- [self, x]
    (def h1 (F.relu (self.l1 x)))
    (def h2 (F.relu (self.l2 h1)))
    (def y (self.l3 h2))
    y))

In [25]:
(def model (L.Classifier MLP 100 10))  ;; the input size, 784, is inferred
(def optimizer (optimizers.SGD))
(optimizer.setup model)

In [26]:
(def updater (training.StandardUpdater train_iter optimizer))
(def trainer (training.Trainer updater [20 "epoch"] :out "result"))

In [27]:
(trainer.run)

Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/calysto_hy/kernel.py", line 98, in do_execute_direct
    eval(code, self.env)
  File "In [27]", line 1, in <module>
  File "/opt/conda/lib/python3.6/site-packages/chainer/training/trainer.py", line 313, in run
    six.reraise(*sys.exc_info())
  File "/opt/conda/lib/python3.6/site-packages/six.py", line 686, in reraise
    raise value
  File "/opt/conda/lib/python3.6/site-packages/chainer/training/trainer.py", line 299, in run
    update()
  File "/opt/conda/lib/python3.6/site-packages/chainer/training/updater.py", line 223, in update
    self.update_core()
  File "/opt/conda/lib/python3.6/site-packages/chainer/training/updater.py", line 234, in update_core
    optimizer.update(loss_func, *in_arrays)
  File "/opt/conda/lib/python3.6/site-packages/chainer/optimizer.py", line 528, in update
    loss = lossfun(*args, **kwds)
  File "/opt/conda/lib/python3.6/site-packages/chainer/links/model/classifier.py", li